In [95]:
import os
import biosapi
import cobrakbase
import logging
import json
from cobrakbase.core import KBaseFBAModel
logger = logging.getLogger(__name__)

In [2]:
CACHE_BASE_FOLDER = '/Users/fliu/workspace/jupyter/python3/annotation-server/data/'
ws_fungi = 'jplfaria:narrative_1510597445008'

In [8]:
fungi_model_list = read_json(CACHE_BASE_FOLDER + '/bios_cache_list_fungi.json')

In [3]:
%run ../../scripts/bios_utils.py
%run ../../notebooks/fungi_manual_curation.py

In [96]:
kbase = cobrakbase.KBaseAPI('UGOG6KLAWTCYI2ASYECYHNIIFTEXGA2J')
bios_live = biosapi.BIOS()

In [5]:
%run ../bios_mock.py
bios = BIOS_MOCK(CACHE_BASE_FOLDER + '/bios_cache_fungi.json')

In [91]:
bios_cache = read_json(CACHE_BASE_FOLDER + '/bios_cache_fungi.json')

In [92]:
bios_cache.keys()

dict_keys(['iMM904', 'iJDZ836', 'iAL1006', 'iWV1213', 'iCT646', 'iJO1366', 'iCac802', 'iAF1260', 'iML1515', 'iMA871', 'yeast_6.06', 'iOD907', 'iLC915', 'yeast_7.6', 'iSS884', 'iNL895', 'iRL766', 'iTO977', 'iJL1454', 'iNX804', 'iWV1314'])

In [94]:
for o in fungi_model_list:
    model_id = o['entry']
    if not model_id in bios_cache:
        cmp = None
        spi = None
        rxn = None
        try:
            cmp = bios_live.get_model_compartments(model_id)
            spi = bios_live.get_model_species(model_id)
            rxn = bios_live.get_model_reactions(model_id)
        except:
            pass
        if not cmp == None and not spi == None and not rxn == None:
            bios_cache[model_id] = {
                'cmp' : cmp, 
                'spi' : spi, 
                'rxn' : rxn
            }
            print(o['entry'])
        else:
            print('!', o['entry'])

In [25]:
fbamodel = KBaseFBAModel(kbase.get_object('iJL1454', ws_fungi))

In [74]:
import math

def extract_model_to_bios(fbamodel):
    cmps = []
    spis = []
    rxns = []
    cmp = {}
    for o in fbamodel.get_compartments():
        cmp[o['id']] = o['label']
        ccmp = {
            'major_label': 'ModelCompartment',
            'name': o['label'],
            'id': o['label']
        }
        cmps.append(ccmp)
    for o in fbamodel.metabolites:
        #print(o, o.compartment, cmp[o.compartment])
        spi = {
            'major_label': 'MetaboliteSpecie',
            'id': o.id,
            'compartment': cmp[o.compartment],
            'name': o.name,
        }
        spis.append(spi)

    for o in fbamodel.reactions:
        #print(o.data)

        r = []
        l = []
        for id in o.stoichiometry:
            v = o.stoichiometry[id]
            if v < 0:
                l.append([id, -9999, math.fabs(v)])
            elif v > 0:
                r.append([id, -9999, math.fabs(v)])


        rxn = {
            'major_label': 'ModelReaction',
            'bios_stoichiometry': { 'r': r, 'l': l},
            'bios_id': -1,
            'name': o.name,
            'id': o.id
        }
        rxns.append(rxn)
    return cmps, spis, rxns


In [80]:
fbamodel = KBaseFBAModel(kbase.get_object('iJL1454', ws_fungi))
cmps, spis, rxns = extract_model_to_bios(fbamodel)
bios_cache['iJL1454'] = {
    'cmp' : cmps, 
    'spi' : spis, 
    'rxn' : rxns
}

In [81]:
fbamodel = KBaseFBAModel(kbase.get_object('iNX804', ws_fungi))
cmps, spis, rxns = extract_model_to_bios(fbamodel)
bios_cache['iNX804'] = {
    'cmp' : cmps, 
    'spi' : spis, 
    'rxn' : rxns
}

In [82]:
fbamodel = KBaseFBAModel(kbase.get_object('iWV1314', ws_fungi))
cmps, spis, rxns = extract_model_to_bios(fbamodel)
bios_cache['iWV1314'] = {
    'cmp' : cmps, 
    'spi' : spis, 
    'rxn' : rxns
}

In [88]:
bios_cache['iWV1314']['rxn'][0]

{'major_label': 'ModelReaction',
 'bios_stoichiometry': {'r': [['M_NADP', -9999, 1.0], ['M_VEML', -9999, 1.0]],
  'l': [['M_138THN', -9999, 1.0], ['M_NADPH', -9999, 1.0]]},
 'bios_id': -1,
 'name': 'Tetrahydroxynaphthalene reductase',
 'id': 'R_r1888'}

In [89]:
write_json(bios_cache, CACHE_BASE_FOLDER + '/bios_cache_fungi.json')

In [150]:
MODEL_RXN_MAPPING = None
with open(CACHE_BASE_FOLDER + 'rxn_mapping_cache3.json', 'r') as f:
    MODEL_RXN_MAPPING = json.loads(f.read())

In [144]:
MODEL_RXN_MAPPING_v3 = MODEL_RXN_MAPPING

In [136]:
model_cpd_mapping = {}

In [151]:
model_rxn_mapping = {}

In [137]:
MODEL_CPD_MAPPING.keys()

dict_keys(['iAL1006', 'iJDZ836', 'iMM904'])

In [152]:
for k in MODEL_RXN_MAPPING:
    model_rxn_mapping[k] = MODEL_RXN_MAPPING[k]

In [154]:
for o in l:
    model_id = o['entry']
    if not model_id in model_cpd_mapping or not model_id in model_rxn_mapping:
        print(model_id, model_id in model_cpd_mapping, model_id in model_rxn_mapping)

iWV1314 False False
iMA871 True False
iJDZ836 True False
iNL895 True False
iJL1454 False False
iNX804 False False
iWV1213 False False
iOD907 False False


In [34]:
env_ws = 'filipeliu:narrative_1575329472745'
pangenome = kbase.get_object('FungalTemplateAndRepresentativeSetOrthosV0', env_ws)

In [36]:
env_ws = 'janakakbase:narrative_1570052138482'
pangenome = kbase.get_object('FungalTemplate13GenomeOrthologs-OrthoMCL', env_ws)

In [170]:
env_ws = 'janakakbase:narrative_1570052138482'
pangenome = kbase.get_object('FungalOrthos', env_ws)

In [181]:
valid_genomes = set()
for model_id in found:
    data = bios_model_dict[model_id]
    if not 'bios_refseq' in data:
        print('!', model_id)
    else:
        valid_genomes.add(data['bios_refseq']['entry'])
len(valid_genomes)

14

In [171]:
ACTIVE_GENOMES = ['GCF_000209165.1',
 'GCF_000182925.2',
 'GCF_000002525.2',
 'GCF_000184455.2',
 'GCF_000149615.1',
 'GCF_000226395.1',
 'GCF_000002515.2',
 'GCF_000027005.1',
 'GCF_000006335.2',
 'GCF_000146045.2',
 'GCF_000091025.4',
 'GCF_000002545.3',
 'Mucor_circinelloides_CBS277.49_v2.0',
 'GCF_000002855.3']

In [172]:
active_genomes = ACTIVE_GENOMES

ref_to_genome = {}

for genome_ref in pangenome['genome_refs']:
    info = kbase.get_object_info_from_ref(genome_ref)
    if info.id + '.json' in os.listdir(CACHE_BASE_FOLDER + '/cache/genomes'):
        print(genome_ref, info.id)
    else:
        genome_data = kbase.get_object(info.id, info.workspace_id)
        write_json(genome_data, CACHE_BASE_FOLDER + '/cache/genomes/' + info.id + '.json', True)
    genome_data = read_json(CACHE_BASE_FOLDER + '/cache/genomes/' + info.id + '.json')
    genome = cobrakbase.core.KBaseGenome(genome_data)
    print(info.id, info.workspace_id, genome.data['scientific_name'])
    ref_to_genome[genome_ref] = genome

49164/6/1 GCF_000226395.1
GCF_000226395.1 janakakbase:narrative_1570052138482 Penicillium rubens Wisconsin 54-1255
49164/5/1 GCF_000182925.2
GCF_000182925.2 janakakbase:narrative_1570052138482 Neurospora crassa OR74A
49164/4/1 GCF_000146045.2
GCF_000146045.2 janakakbase:narrative_1570052138482 Saccharomyces cerevisiae S288c
Encephalitozoon_hellem_ATCC_50504 janakakbase:narrative_1570052138482 Encephalitozoon hellem ATCC 50504
Schizophyllum_commune_H4-8 janakakbase:narrative_1570052138482 Schizophyllum commune H4-8


In [173]:
genome_id_to_ref = {}

for ref in ref_to_genome:
    genome = ref_to_genome[ref]
    genome_id_to_ref[genome.id] = ref
    print(ref, genome.id, genome.data['scientific_name'])

49164/6/1 GCF_000226395.1 Penicillium rubens Wisconsin 54-1255
49164/5/1 GCF_000182925.2 Neurospora crassa OR74A
49164/4/1 GCF_000146045.2 Saccharomyces cerevisiae S288c
49164/3/1 GCF_000277815.2 Encephalitozoon hellem ATCC 50504
49164/2/1 GCF_000143185.1 Schizophyllum commune H4-8


In [177]:
%run ../annotation_ortholog.py
annotation_orth = AnnotationOrtholog(pangenome, bios)
annotation_orth.ref_to_genome = ref_to_genome
annotation_orth.genome_id_to_ref = genome_id_to_ref

In [43]:
#bios.model_data[model_id].keys()

In [272]:
rxn = bios_get_model_reaction(model_id, rxn_id)
print(rxn)

In [258]:
fbamodel = KBaseFBAModel(kbase.get_object('iAL1006_KBase3', fungi_ws))

In [259]:
model_id = 'iAL1006'
#model_rxn_mapping[model_id] = {}

In [261]:
for r in fbamodel.reactions:
    seed_ids = []
    sid = r.get_original_id()
    if 'dblinks' in r.data and 'ModelSeedReaction' in r.data['dblinks']:
        seed_ids = r.data['dblinks']['ModelSeedReaction']
        #print(r.id, r.data['dblinks']['ModelSeedReaction'])
    elif 'dblinks' in r.data and 'Seed' in r.data['dblinks']:
        seed_ids = r.data['dblinks']['Seed']
    if len(seed_ids) > 0:
        if model_id in model_rxn_mapping and not sid in model_rxn_mapping[model_id]:
            model_rxn_mapping[model_id][sid] = []
        if model_id in model_rxn_mapping and sid in model_rxn_mapping[model_id]:
            if len(model_rxn_mapping[model_id][sid]) == 0:
                print(sid, ' -> ', seed_ids)
                model_rxn_mapping[model_id][sid] = seed_ids
            elif not set(model_rxn_mapping[model_id][sid]) == set(seed_ids):
                print(sid, ' -> ', model_rxn_mapping[model_id][sid], '->', seed_ids)
                model_rxn_mapping[model_id][sid] = seed_ids

In [293]:
annotation_orth.models['iAL1006'] = {}
annotation_orth.models['iMM904'] = {}

In [296]:
rxn = bios_get_model_reaction(model_id, rxn_id)
print(rxn_id, get_rxn_compartment(rxn, model_id))

R_xylanIN ['C_b', 'C_e']


In [304]:
model_id = 'iAL1006'
model_rxn_mapping['iMM904']['R_PYK']

['rxn35242']

In [53]:
def get_reaction_mapping(model_id, rxn_id):
    if model_id in model_rxn_mapping and rxn_id in model_rxn_mapping[model_id]:
        return model_rxn_mapping[model_id][rxn_id]
get_reaction_mapping('iAL1006', 'R_r0003')

[]

In [315]:
for o in kbase.list_objects(fungi_ws):
    if 'KBaseFBA.FBAModel' in o[2]:
        print(o[1])
    break

In [30]:
def synch_check(fbamodel, bios_data, model_id):
    spi_miss = set()
    rxn_miss = set()
    for m in fbamodel.metabolites:
        sid = m.get_original_id()
        s = bios_get_model_species(bios, model_id, sid)
        if s == None:
            spi_miss.add(m.id)
    for o in fbamodel.reactions:
        sid = o.get_original_id()
        s = bios_get_model_reaction(bios, model_id, sid)
        if s == None:
            rxn_miss.add(m.id)
    print(spi_miss, rxn_miss)
synch_check(kbase_models['iAL1006'], bios, 'iAL1006')

set() set()


In [328]:
fbamodel = KBaseFBAModel(kbase_models['iMM904_KBase3'])

In [333]:
bios_cache['iMM904']['spi']

dict_keys(['cmp', 'spi', 'rxn'])

In [48]:
model_rxn_mapping = None
with open(CACHE_BASE_FOLDER + 'rxn_mapping_cache3.json', 'r') as f:
    model_rxn_mapping = json.loads(f.read())
model_cpd_mapping = None
with open(CACHE_BASE_FOLDER + 'cpd_mapping_cache3.json', 'r') as f:
    model_cpd_mapping = json.loads(f.read())

In [200]:
def get_reactions_by_compartment(aaaaaaaaa, seed_rxn_id, compartment):
    pass

In [84]:
def get_model_rxn_grp(aaaaaaaaa, bios, seed_rxn_id, compartment = None, standard_compartment = True):
    model_rxn_grp = {}
    for model_id in aaaaaaaaa.models:
        model_rxn_grp[model_id] = {}
        if model_id in model_rxn_mapping:
            for rxn_id in model_rxn_mapping[model_id]:
                if len(model_rxn_mapping[model_id][rxn_id]) > 0:
                    rxn = bios_get_model_reaction(bios, model_id, rxn_id)
                    cmp = get_rxn_compartment(bios, rxn, model_id)
                    seed_id = get_reaction_mapping(model_id, rxn.id)
                    if seed_rxn_id in seed_id:
                        if compartment == None:
                            kbase_rxn = get_kbase_reaction(aaaaaaaaa, model_id, rxn.id)
                            if not kbase_rxn == None:
                                model_rxn_grp[model_id][rxn.id] = kbase_rxn
                            else:
                                print('!!!!')
                        else:
                            model_rxn_grp[model_id][rxn.id] = rxn
                #print(rxn_id, get_rxn_compartment(rxn, model_id))
    return model_rxn_grp
model_rxn_grp = get_model_rxn_grp(annotation_orth, bios, 'rxn00148')

In [ ]:
annotation_orth.get_genes

In [73]:
model_rxn_grp['iCT646']['R_PYK']

In [146]:
model_gene_reaction = {}

In [208]:
def get_orthologs_from_seed_rxn_id2(aaaaaaaaa, bios, seed_rxn_id, compartment = None, standard_compartment = True):
    model_rxn_grp = get_model_rxn_grp(aaaaaaaaa, bios, seed_rxn_id, compartment, standard_compartment)
    genome_match = {}

    all_orthologs = set()
    matched_orthologs = set()
    for model_id in model_rxn_grp:
        genome_id = aaaaaaaaa.model_to_genome[model_id]
        if genome_id in aaaaaaaaa.genome_id_to_ref:
            if not genome_id in genome_match:
                genome_match[genome_id] = {}
            genome_match[genome_id][model_id] = []
            genome = aaaaaaaaa.ref_to_genome[aaaaaaaaa.genome_id_to_ref[genome_id]]
            for rxn_id in model_rxn_grp[model_id]:
                gpr_exp = model_rxn_grp[model_id][rxn_id].data['imported_gpr']
                genes = aaaaaaaaa.get_genes(gpr_exp)
                
                logger.debug("%s %s %s", genome_id, gpr_exp, genes)
                #print(genome_id, gpr_exp, genes)
                exp_match = {
                    'rxn_id' : rxn_id,
                    'gpr' : model_rxn_grp[model_id][rxn_id].data['imported_gpr'],
                    'genes' : {}
                }
                
                for gene_id in genes:
                    features = list(filter(lambda x : x['id'] == gene_id, genome.features))
                    if len(features) == 1:
                        ortholog = aaaaaaaaa.get_ortholog(genome_id, gene_id)
                        matched_orthologs.add((genome_id, gene_id, ortholog['id']))
                        for o in ortholog['orthologs']:
                            o_genome_id = aaaaaaaaa.ref_to_genome[o[2]].id
                            if o_genome_id in aaaaaaaaa.model_to_genome.values():
                                all_orthologs.add((o_genome_id, o[0], ortholog['id']))
                        gene_function = '?' if not 'function' in features[0] else features[0]['function']
                        print(features[0]['id'], gene_function, ortholog['id'])
                        exp_match['genes'][gene_id] = {
                            'ortholog_id' : ortholog['id'],
                            'function' : gene_function
                        }
                    else:
                        print('!', gene_id, len(features))
                logger.debug("%s", exp_match)
                genome_match[genome_id][model_id].append(exp_match)
                #print(rxn_id, model_rxn_grp[model_id][rxn_id], gpr_exp)
    return all_orthologs, matched_orthologs, genome_match

all_orthologs, matched_orthologs, genome_match = get_orthologs_from_seed_rxn_id2(annotation_orth, bios, 'rxn00148')

YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
Pc18g06000 hypothetical protein cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614
YOR347C pyruvate kinase PYK2 cluster614
YAL038W pyruvate kinase CDC19 cluster614


In [ ]:
'KLLA0F23397g'

In [180]:
ortholog = annotation_orth.get_ortholog('GCF_000146045.2', 'YAL038W')
for o in ortholog['orthologs']:
    o_genome_id = annotation_orth.ref_to_genome[o[2]].id
    print(o, o_genome_id, o_genome_id in annotation_orth.model_to_genome.values())

['YAL038W', 30, '49164/4/1'] GCF_000146045.2 True
['YOR347C', 5854, '49164/4/1'] GCF_000146045.2 True
['Pc18g06000', 8904, '49164/6/1'] GCF_000226395.1 True
['NCU06075', 9789, '49164/5/1'] GCF_000182925.2 True
['EHEL_090610', 1328, '49164/3/1'] GCF_000277815.2 False
['SCHCODRAFT_73013', 1702, '49164/2/1'] GCF_000143185.1 False


In [209]:

res = process_data2(annotation_orth, all_orthologs, genome_match)
res

{'gene_reaction': {'GCF_000146045.2': {'YAL038W': ['r_0962'],
   'YOR347C': ['r_0962']},
  'GCF_000226395.1': {'Pc18g06000': ['R_r0016']},
  'GCF_000182925.2': {'NCU06075': ['PEPDEPHOS__45__RXN__45__R2L']},
  'GCF_000277815.2': {'EHEL_090610': []},
  'GCF_000143185.1': {'SCHCODRAFT_73013': []}},
 'orthologs': {'cluster614': {'GCF_000146045.2': {'YAL038W': 'pyruvate kinase CDC19',
    'YOR347C': 'pyruvate kinase PYK2'},
   'GCF_000226395.1': {'Pc18g06000': 'hypothetical protein'},
   'GCF_000182925.2': {'NCU06075': 'pyruvate kinase'},
   'GCF_000277815.2': {'EHEL_090610': '?'},
   'GCF_000143185.1': {'SCHCODRAFT_73013': '?'}}},
 'genomes': {'GCF_000146045.2': {'name': 'Saccharomyces cerevisiae S288c',
   'source': {'iMM904': [{'rxn_id': 'R_PYK',
      'gpr': '(YAL038W or YOR347C)',
      'genes': {'YOR347C': {'ortholog_id': 'cluster614',
        'function': 'pyruvate kinase PYK2'},
       'YAL038W': {'ortholog_id': 'cluster614',
        'function': 'pyruvate kinase CDC19'}}}],
    'iTO9

In [196]:
ortholog_data = list(filter(lambda x : x['id'] == 'cluster614', annotation_orth.ortho['orthologs']))[0]
for o in ortholog_data['orthologs']:
    print(o)
    genome_ref = o[2]
    genome = annotation_orth.ref_to_genome[genome_ref]
    print(genome.id)

['YAL038W', 30, '49164/4/1']
GCF_000146045.2
['YOR347C', 5854, '49164/4/1']
GCF_000146045.2
['Pc18g06000', 8904, '49164/6/1']
GCF_000226395.1
['NCU06075', 9789, '49164/5/1']
GCF_000182925.2
['EHEL_090610', 1328, '49164/3/1']
GCF_000277815.2
['SCHCODRAFT_73013', 1702, '49164/2/1']
GCF_000143185.1


In [166]:
def add_gene_reactions(res):

            
        #print(res['orthologs'][o][genome_id])
res['gene_reaction']

{'GCF_000091025.4': {'AGOS_ADR368W': {}},
 'GCF_000184455.2': {'AO090005001556': {}},
 'GCF_000002515.2': {'KLLA0F23397g': {}},
 'GCF_000209165.1': {'PICST_83166': {}},
 'GCF_000006335.2': {'CTRG_01460': {}},
 'GCF_000002545.3': {'CAGL0M12034g': {}, 'CAGL0E05610g': {}},
 'GCF_000027005.1': {'PAS_chr2-1_0769': {}},
 'GCF_000182925.2': {'NCU06075': {}},
 'GCF_000149615.1': {'ATEG_03685': {}},
 'GCF_000226395.1': {'Pc18g06000': {}},
 'Mucor_circinelloides_CBS277.49_v2.0': {'156910': {}},
 'GCF_000002525.2': {'YALI0F09185g': {}},
 'GCF_000146045.2': {'YAL038W': {}, 'YOR347C': {}}}

In [97]:
env_ws = 'filipeliu:narrative_1575329472745'
ortho = kbase.get_object('FungalTemplateAndRepresentativeSetOrthosV0', env_ws)

In [101]:
for genome_ref in ortho['genome_refs']:
    info = kbase.get_object_info_from_ref(genome_ref)
    copy_object(kbase.ws_client, info.id, info.workspace_id, 
                info.id, env_ws)
    #genome_data = read_json(annotation_path + info.id + '.json')
    #genome_data = kbase.get_object(info.id, info.workspace_id)
    #genome = cobrakbase.core.KBaseGenome(genome_data)
    print(info.id, info.workspace_id)
    #ref_to_genome[genome_ref] = genome

Rhizophagus_irregularis janakakbase:narrative_1570052138482
Rhodotorula_toruloides_NP11 janakakbase:narrative_1570052138482
GCF_000002515.2 janakakbase:narrative_1570052138482
Tuber_melanosporum_Mel28 janakakbase:narrative_1570052138482
Ustilago_maydis_521 janakakbase:narrative_1570052138482
GCF_000149615.1 janakakbase:narrative_1570052138482
Cryptococcus_neoformans_var._grubii_H99 janakakbase:narrative_1570052138482
Piromyces_finnis janakakbase:narrative_1570052138482
GCF_000209165.1 janakakbase:narrative_1570052138482
GCF_000002525.2 janakakbase:narrative_1570052138482
Mucor_circinelloides_CBS277.49_v2.0 janakakbase:narrative_1570052138482
Mucor_circinelloides_f_lusitanicus_CBS277_49 janakakbase:narrative_1570052138482
GCF_000027005.1 janakakbase:narrative_1570052138482
Aspergillus_nidulans janakakbase:narrative_1570052138482
Laccaria_bicolor janakakbase:narrative_1570052138482
GCF_000002545.3 janakakbase:narrative_1570052138482
GCF_000091025.4 janakakbase:narrative_1570052138482
Rhi

In [99]:
def copy_object(wclient, from_id, from_ws, to_id, to_ws):
    params = {
        'from' : {'name' : from_id, 'workspace' : from_ws},
        'to' : {'name' : to_id, 'workspace' : to_ws}
    }
    return wclient.copy_object(params)